## Coursera Capstone assignment - Toronto neighbourhood Clustering & Segmenting - Part 1

This notebook is created as part of Corusera Capstone project. This notebook is created as part of capstone project  assignment - 'Clustering & Segmenting Toronto city data'. This notebook implements the following:

1. Extract the post codes & Neighbourhood data correspoding to various bouroughs in Toronto, ontario from a wikipedia page using python library - **BeautifulSoup**
    
2. Store the extracted data into a dataframe with columns as below:
    - Postal Code
    - Borough
    - Neighbourhood
    

### Installing BeautifulSoup library

In [4]:
!conda install -c conda-forge beautifulsoup4  --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.6.7

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /Users/vivekvenudasan/anaconda3

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.7.1       |        py36_1001         140 KB  conda-forge
    soupsieve-1.8              |           py36_0         102 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         242 KB

The following NEW packages will be INSTALLED:

    soupsieve:      1.8-py36_0           conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py36h72d3c9f_1             --> 4.7.1-py36_1001 conda-forge


beautifulsoup4 4.

### Import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#import BeautifulSoup library for parsing wikipedia page
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


### Extract wikipedia page using requests library

In [3]:
wiki_page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_doc =  requests.get(wiki_page)
print('Done')

Done


### Use Beautiful soup to parse and store the extracted wikipage into a dictionary and then finally store it into toronto neighborhoods dataframe (target)

In [4]:
wikipage = BeautifulSoup(html_doc.text, 'html.parser') #read the extracted Wikipage using beautifulsoup
table = wikipage.find("table")
i=0
j=0
temp_dict=dict()
for row in table.findAll("tr"): 
    col = row.findAll("td")
    if len(col)>0:
        pcode = col[0].get_text().strip('\n')
        borough = col[1].get_text().strip('\n')
        neighbourhood = col[2].get_text().strip('\n')
        dict_key=pcode+'_'+borough
        
        if neighbourhood == 'Not assigned':
            neighbourhood = borough # to handle neighbouhoods that are 'Not assgined' & assign them to borough value in such cases
            
        if borough != 'Not assigned':  #skips all boroughs which are having value = 'Not assigned'
            #group the neighbourhoods belonging to same postal area in a comma separated csv
            if dict_key in temp_dict:
                temp_dict[dict_key]=temp_dict[dict_key]+','+neighbourhood
            else:
                temp_dict[dict_key]=neighbourhood
        else:
            j=j+1
        i=i+1
#priting howmany entries skipped        
text = 'Skipped {} boroughs out of {} entries as they are <Not assigned>\n'.format(j,i)
print(text)

#convert dictionary to a temporary dataframe
temp_df = pd.DataFrame(temp_dict.items(),columns=['key','Neighbourhood'])
#Split the key column to separate columns
temp_df[['Postal Code','Borough']] = temp_df['key'].str.split('_', n=1, expand=True)
# drop the key column
temp_df.drop('key',axis=1,inplace=True)

#Rearrange the columns and store the data to the base dataframe for toronto neighbourhoods
toronto_nhood_data = temp_df[['Postal Code','Borough','Neighbourhood']]
print("Toronto neighbourhoods dataframe created!!")


Skipped 77 boroughs out of 289 entries as they are <Not assigned>

Toronto neighbourhoods dataframe created!!


### Print the shape & head of the target dataframe

In [5]:
print('Grouped neighbourhoods example:{}\n'.format(toronto_nhood_data[toronto_nhood_data['Postal Code']=='M5A']))
print('Not assigned neighbouhoods handle example:{}\n'.format(toronto_nhood_data[toronto_nhood_data['Postal Code']=='M7A']))

print('Dataframe shape:',toronto_nhood_data.shape,'\n')

toronto_nhood_data.head(15)

Grouped neighbourhoods example:  Postal Code           Borough             Neighbourhood
2         M5A  Downtown Toronto  Harbourfront,Regent Park

Not assigned neighbouhoods handle example:  Postal Code       Borough Neighbourhood
4         M7A  Queen's Park  Queen's Park

Dataframe shape: (103, 3) 



,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"
